# Libraries

In [1]:
import jupyternotify
import pandas as pd
import requests
import calendar
import json
import os
from urllib.request import urlopen
from datetime import timedelta
from datetime import datetime
from zipfile import ZipFile
from io import BytesIO
from time import sleep
import pyodbc
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode

ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

# Paramaters

In [3]:
country = 'Ecuador'

path = '../../01ISV/Params/'
for json_file in [file for file in os.listdir(path) if file.endswith('.json')]:  
    with open(path + json_file, encoding='utf8') as f:
        globals()[json_file.split('.')[0].split('_')[1]] = json.load(f)

for json_file in [file for file in os.listdir(path + country + '/') if file.endswith('.json')]:
    with open(path + country + '/' + json_file, encoding='utf8') as f:
        globals()[json_file.split('.')[0]] = json.load(f)

for sql_file in [file for file in os.listdir(path + country + '/') if file.endswith('.sql')]:
    globals()[sql_file.split('.')[0]] = query = open(path + country + '/' + sql_file, encoding='utf8')

## Sales

In [4]:
sales['headers_sales']['Authorization'] = 'ISVToken ' + tokens[country]
body_sales = sales['body_sales']
body_sales['views'] = sales['views'][country]
body_sales['hierarchy'] = sales['hierarchy_sales'][country]
body_sales['view_type'] = 'semana'

## Stores

In [5]:
url_stores = stores['url_stores']
headers_stores = stores['headers_stores']
headers_stores['Authorization'] = 'ISVToken ' + tokens[country]

## Stock

In [6]:
url_stock = stocks['url_stock']
headers_stock = stocks['headers_stock']
headers_stock['Authorization'] = 'ISVToken ' + tokens[country]
hierarchy_stock = stocks['hierarchy_stock']
body_stock = stocks['body_stock']
body_stock['hierarchy'] = hierarchy_stock

## DWH

In [7]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=' + servers_dbs['server'] + ';'
                     'Database=' + servers_dbs['database1'] + ';'
                     'Trusted_Connection=yes;')

conn2 = pyodbc.connect('Driver={SQL Server};'
                     'Server=' + servers_dbs['server'] + ';'
                     'Database=' + servers_dbs['database2'] + ';'
                     'Trusted_Connection=yes;')

# Load data

## DWH

### Tmpid

In [8]:
df_tmpid = pd.read_sql(query_dates.read(), conn1)
df_tmpid['TmpFecha'] = df_tmpid['TmpFecha'].astype(str).copy()
df_tmpid.shape

(7304, 2)

### ProPstID

In [9]:
df_ppst = pd.read_sql(query_ppst.read(), conn2)
df_ppst['ProPstCodBarras'] = df_ppst['ProPstCodBarras'].astype(str).copy()
df_ppst.shape

(192, 2)

## Sales

**Semana ISV = Semana Genomma - 1**

In [10]:
weekly_format = lambda number: '0' + str(number) if (number < 10) else str(number)

clean_date = lambda str_date: datetime(int(str_date.split('-')[2]), int(str_date.split('-')[1]), int(str_date.split('-')[0]))

clean_numbers = lambda str_numb: float(str(str_numb).replace('.','').replace(',',''))/1000000

In [ ]:
def check_status(status):
    if status != 200:
        

In [29]:
def unpack_file(response):
    file = urlopen(response.json()['download_url'])
    zip_file = ZipFile(BytesIO(file.read()))
    df = pd.read_csv(zip_file.open(zip_file.namelist()[0]), encoding='latin-1', sep=';')
    return df

In [10]:
def download_sales(num_week, url, header, body):
    body["dates"] = ["2020-W" + weekly_format(num_week)]
    resp_sales = requests.post(url, data=json.dumps(body), headers=header)
    if resp_sales.status_code != 200:
        print("Oh, oh, adventurous, problems  in week " + str(num_week) + " :S")
        pass
    else:
        df = unpack_data(resp_sales)
    return df

In [11]:
def clean_sales(data):    
    data['Cód. Cadena'] = data['Cód. Cadena'].astype(str).copy()
    data['EAN'] = data['EAN'].astype(str).copy()    
    for col in ['Unidades', 'Costos B2B']:
        data[col] = data[col].map(clean_numbers).copy()
    data['ID_SaSt'] = data['EAN'].astype(str) + data['Cód. Cadena'].astype(str) + data['Cadena'] + data['Local']  
    return data

## Stores

In [12]:
def download_stores(url, header):
    resp_stores = requests.get(url, headers=header)
    if resp_stores.status_code != 200:
        print("Oh, oh, problems body")
    else:
        df = pd.read_excel(resp_stores.json()['url'], sep='\t')
        ind_min = df[df.iloc[:,7].notnull()].index.min()
        cols = df.iloc[9,:].tolist()
        data = df.loc[ind_min + 2:].copy()
        data.reset_index(drop=True, inplace=True)
        data.rename(dict(zip(df.columns.tolist(), cols)), inplace=True, axis=1)
    return data

## Stock

In [13]:
def download_stock(dates, chain, url, header, body):
    df_stock = pd.DataFrame()
    hierarchy_stock['Cadena'] = chain
    for date in dates:
        body["dates"] = [date.strftime('%Y-%m-%d')]
        resp_stock = requests.post(url, data=json.dumps(body), headers=header)
        if resp_stock.status_code != 200:
            print("Oh, oh, adventurous, problems in " + date.strftime('%d/%m/%Y') + ' :S')
            pass
        else:
            resp_page = urlopen(resp_stock.json()['download_url'])
            zip_file = ZipFile(BytesIO(resp_page.read()))
            df = pd.read_csv(zip_file.open(zip_file.namelist()[0]), encoding='latin-1', sep=';')
            df_stock = pd.concat([df_stock, df], axis=0)
    df_stock.reset_index(drop=True, inplace=True)
    return df_stock

In [14]:
def iterate_stock(dates, chains):
    df = pd.DataFrame()
    for date in dates:
        df = pd.concat([df, download_stock(dates=[date], chain=[], url=url_stock, header=headers_stock, body=body_stock)], axis=0)
        for chain in chains:
            chain_stock = pd.DataFrame()
            days = 0
            if chain not in df['Cadena'].unique():                
                while chain_stock.shape[0] == 0:
                    days += 1            
                    chain_stock = download_stock(dates=[date - timedelta(days=days)], chain=[chain], url=url_stock, header=headers_stock, body=body_stock)
                    sleep(17)
                    if days > 7:
                        break
            try:
                df = pd.concat([df, chain_stock[chain_stock['Cadena'] == chain]], axis=0)
            except:
                pass
    df.reset_index(inplace=True, drop=True)
    return df

### Clean data

In [15]:
def clean_stock(data):
    data_clean = data[['Fechas', 'EAN', 'Cadena', 'Sub Cadena', 'Local', 'Stock Locales en Unidades', 'Stock CD en Unidades', 'Stock en Tránsito en Unidades', 'Stock Total en Unidades']].copy()
    for col in ['Stock Locales en Unidades', 'Stock CD en Unidades', 'Stock en Tránsito en Unidades', 'Stock Total en Unidades']:
        data_clean[col] = data_clean[col].map(clean_numbers)
    data_clean['ID_SaSt'] = data['EAN'].astype(str) + data['Cód. Cadena'].astype(str) + data['Cadena'] + data['Local']
    data_clean['ID_StSt'] = data['Cadena'] + data['Sub Cadena'] + data['Local']
    return data_clean

# Join

In [16]:
fill = lambda col1, col2: col1 if pd.isnull(col2) else (col2 if pd.isnull(col1) else col1)

In [17]:
def relate_dates(df1, df2):
    d = {}
    for date1 in df1:
        for date2 in df2:
            d11 = clean_date(date1.split(' ')[2])
            d12 = clean_date(date1.split(' ')[-1])
            d2 = clean_date(date2)
            if d11 <= d2 <= d12:
                d[d2.strftime('%d-%m-%Y')] = d12.strftime('%d-%m-%Y')
    return d

In [18]:
def join_data(sales, stock, stores, df_ppst):
        join = pd.merge(sales, stock, on='ID_SaSt', how='outer')
        join['Cadena'] = join['Cadena_y'].combine(join['Cadena_x'], fill)
        join['Sub Cadena'] = join['Sub Cadena_y'].combine(join['Sub Cadena_x'], fill)
        join['Local'] = join['Local_y'].combine(join['Local_x'], fill)
        join['EAN'] = join['EAN_y'].combine(join['EAN_x'], fill)
        join['Fecha'] = join['Fecha_y'].combine(join['Fecha_x'], fill)
        join.drop(['Cadena_x', 'Cadena_y', 'Sub Cadena_x', 'Sub Cadena_y', 'Local_x', 'Local_y', 'EAN_x', 'EAN_y', 'Fecha_x', 'Fecha_y', 'ID_SaSt', 'ID_StSt'], axis=1, inplace=True)
        join.fillna(0, inplace=True)
        join2 = pd.merge(join, stores, on='Local', how='left')
        join2.rename({'EAN':'ProPstCodBarras'}, axis=1, inplace=True)
        join2 = join2[join2['ProPstCodBarras'] != 'No Definido'].copy()
        join2['ProPstCodBarras'] = join2['ProPstCodBarras'].astype('int64').astype(str)
        join3 = pd.merge(join2, df_ppst, on='ProPstCodBarras', how='left')
        return join3

In [19]:
def status(df1, df2, df3):
    cols = ['Unidades', 'Costos B2B', 'Stock Locales en Unidades',
           'Stock CD en Unidades', 'Stock en Tránsito en Unidades',
           'Stock Total en Unidades']

    for col in cols:
        try:
            sum_equals = (df1[col].sum() == df2[col].sum())
            if sum_equals == False:
                diff = (df1[col].sum() - df2[col].sum())
        except:
            sum_equals = df1[col].sum() == df3[col].sum()
            if sum_equals == False:
                diff = (df1[col].sum() - df3[col].sum())
        if sum_equals == False:
            print(sum_equals, " - ", col, '  -  Difference:', diff)
        else:
            print(sum_equals, " - ", col)

Aquí es donde hace casi toda la magia:

In [ ]:
%%notify -m "¡Descarga de ECUADOR lista!"
%%time
# Semana ISV = Semana Genomma - 1
final = pd.DataFrame()
df_sales = {}
data_sales = {}
df_stock = {}
data_stock = {}
#df_stores = pd.read_excel('../../1Data/2Catalogue/SucID_41_43.xlsx')
df_stores = download_stores(url_stores, headers_stores)
stores = df_stores[['Local', 'Suc. ID']].copy()
weeks = [datetime.today().isocalendar()[1] - i for i in range(3, 1, -1)]
for week in weeks:
    ## Download the sales data
    df_sales[str(week)] = download_sales(week, url=sales['url_sales'], body=body_sales, header=sales['headers_sales'])
    ## Clean it
    data_sales[str(week)] = clean_sales(df_sales[str(week)])
    ## Download the stock data
    df_stock[str(week)] = iterate_stock([clean_date(df_sales[str(week)]['Semanas'].unique()[0][-10:])], df_sales[str(week)]['Cadena'].unique().tolist())       
    ## Clean it
    data_stock[str(week)] = clean_stock(df_stock[str(week)])
    ## Verify the stock data
    print("Total de stock a la semana " + str(week + 1) + ":", "\n")
    print(pd.pivot_table(data_stock[str(week)], index=['Cadena'], columns=['Fechas'], values=['Stock Locales en Unidades'], aggfunc='sum'))
    ## Assign a date to sales data and stock data
    dict_dates = relate_dates(data_sales[str(week)]['Semanas'].unique(), data_stock[str(week)]['Fechas'].unique())
    ## Format the date to sales data
    sellout = data_sales[str(week)][['Semanas', 'Cadena', 'Sub Cadena', 'Local','EAN','Unidades', 'Costos B2B', 'ID_SaSt']].copy()
    sellout['Fecha'] = sellout['Semanas'].apply(lambda x: x[-10:]).copy()
    sellout.drop(['Semanas'], axis=1, inplace=True)
    ## Format the date to stock data
    stock = data_stock[str(week)].copy()
    stock['Fecha'] = data_stock[str(week)]['Fechas'].map(dict_dates)
    stock.drop(['Fechas'], axis=1, inplace=True)
    ## Merge sales, stock and stores data
    all_data = join_data(sellout, stock, stores, df_ppst)
    ## We verify some fields
    print('\n', 'Missings por columna:\n', all_data.isnull().sum())
    print('\n', 'Códigos de Barra sin ProPstID  ', all_data[all_data['ProPstID'].isnull()]['ProPstCodBarras'].unique(), '\n')
    status(all_data, data_stock[str(week)], data_sales[str(week)])
    ## We concant into a single variable
    final = pd.concat([final, all_data], axis=0)
    print('\n\n')
    print('-----------------------------------------------------------------------')
    print('\n\n')
final.reset_index(drop=True, inplace=True)

## Validations

Usualmente cuando las sucursales (_Suc. ID_) no están en ISV, vienen como _"No Definidas"_:

In [40]:
final.shape

(26913, 13)

In [39]:
final['Local'][final['Suc. ID'].astype(str).str.contains('No')].unique()

array(['0 Farmamia - Local genérico'], dtype=object)

In [49]:
final.loc[final['Suc. ID'].astype(str).str.contains('No'), 'Suc. ID'] = '146777'
final.loc[final['Suc. ID'] == '0', 'Suc. ID'] = '146100'

In [55]:
print('min:', final['Suc. ID'].astype('int64').min(), '\n','max:', final['Suc. ID'].astype('int64').max())

min: 34094 
 max: 146777


In [56]:
data_sales.keys()

dict_keys(['42', '43'])

In [57]:
final.columns

Index(['Unidades', 'Costos B2B', 'Stock Locales en Unidades',
       'Stock CD en Unidades', 'Stock en Tránsito en Unidades',
       'Stock Total en Unidades', 'Cadena', 'Sub Cadena', 'Local',
       'ProPstCodBarras', 'Fecha', 'Suc. ID', 'ProPstID'],
      dtype='object')

In [58]:
final['Fecha'].unique()

array(['25-10-2020', '01-11-2020'], dtype=object)

In [59]:
df_stock.keys()

dict_keys(['42', '43'])

# Last Transformation

In [60]:
new_names = {'Unidades':'SoutCantDesp',
             'Stock Locales en Unidades':'SoutCantExist',
             'Stock CD en Unidades':'SoutCantCedis',
             'Stock en Tránsito en Unidades':'SoutCantTrans',
             'Stock Total en Unidades':'SoutCantInv',
             'Costos B2B':'SoutMontoDesp',
             'Stock Locales en Precio Lista':'SoutMontoExist',
             'Stock CD en Precio Lista':'SoutMontoCedis',
             'Stock Total en Precio Lista':'SoutMontoInv',             
             'EAN_x':'ProPstCodBarras',
             'Suc. ID':'SucID',
             'Pro. Pst. ID':'ProPstID'}

final.rename(new_names, inplace=True, axis=1)

In [61]:
final['TmpFecha'] = final['Fecha'].map(clean_date).astype(str)

final = pd.merge(final, df_tmpid, on='TmpFecha', how='left')

In [62]:
final['SoutCantStaple'] = 0
final['SoutMontoTrans'] = 0
final['SoutMontoStaple'] = 0
final['SoutMontoExist'] = 0
final['SoutMontoCedis'] = 0
final['SoutMontoTrans'] = 0
final['SoutMontoStaple'] = 0
final['SoutMontoInv'] = 0
final['SoutMontoCteDesp'] = 0

In [63]:
data_final = pd.pivot_table(final, index=['ProPstID', 'SucID', 'TmpID'], values=['SoutCantDesp', 'SoutCantExist', 'SoutCantCedis', 'SoutCantTrans', 'SoutCantStaple', 'SoutCantInv', 'SoutMontoDesp', 'SoutMontoExist', 'SoutMontoCedis', 'SoutMontoTrans', 'SoutMontoStaple', 'SoutMontoInv', 'SoutMontoCteDesp'], aggfunc='sum').reset_index().copy()

In [64]:
final.shape

(26913, 22)

In [65]:
data_final.shape

(26788, 16)

Veificamos que no haya _Pro. Pst. ID_ en 0 (que sigificaría un error)

In [66]:
data_final.shape

(26788, 16)

In [67]:
data_final.dtypes

ProPstID              int64
SucID                object
TmpID                 int64
SoutCantCedis       float64
SoutCantDesp        float64
SoutCantExist       float64
SoutCantInv         float64
SoutCantStaple        int64
SoutCantTrans       float64
SoutMontoCedis        int64
SoutMontoCteDesp      int64
SoutMontoDesp       float64
SoutMontoExist        int64
SoutMontoInv          int64
SoutMontoStaple       int64
SoutMontoTrans        int64
dtype: object

In [68]:
print('min:',data_final['ProPstID'].min(), '\n','max:', data_final['ProPstID'].max())

min: 633 
 max: 8479


In [69]:
print('min:',data_final['SucID'].astype('int64').min(), '\n','max:', data_final['SucID'].astype('int64').max())

min: 34094 
 max: 146777


In [70]:
data_final.isnull().sum()

ProPstID            0
SucID               0
TmpID               0
SoutCantCedis       0
SoutCantDesp        0
SoutCantExist       0
SoutCantInv         0
SoutCantStaple      0
SoutCantTrans       0
SoutMontoCedis      0
SoutMontoCteDesp    0
SoutMontoDesp       0
SoutMontoExist      0
SoutMontoInv        0
SoutMontoStaple     0
SoutMontoTrans      0
dtype: int64

## Last validations

No debe existir duplicados en estas 4 columnas, sino sugeriría que no corrímos bien el *pivot_table*

In [71]:
data_final[['ProPstID', 'SucID', 'TmpID']].duplicated().sum()

0

Lo verifico una vez más XD

In [72]:
data_final['TmpID'].unique()

array([20201025, 20201101], dtype=int64)

In [73]:
new_cols = {'Unidades':'SoutCantDesp',
            'Stock Locales en Unidades':'SoutCantExist',
            'Stock CD en Unidades':'SoutCantCedis',
            'Stock en Tránsito en Unidades':'SoutCantTrans',
            'Stock Total en Unidades':'SoutCantInv',
            'Costos B2B':'SoutMontoDesp'}

In [74]:
data_sales_tot = pd.concat([data_sales['43'], data_sales['42']], axis=0)
data_stock_tot = pd.concat([data_stock['43'], data_stock['42']], axis=0)

In [75]:
for k, v in new_cols.items():
    try:
        status = (data_final[v].sum() == data_stock_tot[k].sum())
        if status == False:
            diff = (data_final[v].sum() - data_stock_tot[k].sum())
    except:
        status = data_final[v].sum() == data_sales_tot[k].sum()
        if status == False:
            diff = (data_final[v].sum() - data_sales_tot[k].sum())
    if status == False:
        print(status, " - ", k, '  ||  Difference:', diff)
    else:
        print(status, " - ", k)

True  -  Unidades
False  -  Stock Locales en Unidades   ||  Difference: 5.820766091346741e-11
True  -  Stock CD en Unidades
True  -  Stock en Tránsito en Unidades
False  -  Stock Total en Unidades   ||  Difference: 5.820766091346741e-11
True  -  Costos B2B


Si hay *SucID* en 0, también sugiere un error:

Vuelvo a verificar, jajaja

In [76]:
data_final[data_final['SucID'].str.contains('No')]

,ProPstID,SucID,TmpID,SoutCantCedis,SoutCantDesp,SoutCantExist,SoutCantInv,SoutCantStaple,SoutCantTrans,SoutMontoCedis,SoutMontoCteDesp,SoutMontoDesp,SoutMontoExist,SoutMontoInv,SoutMontoStaple,SoutMontoTrans


In [77]:
data_upgrade = list(zip(*map(data_final.get, data_final)))

In [78]:
data_final.columns

Index(['ProPstID', 'SucID', 'TmpID', 'SoutCantCedis', 'SoutCantDesp',
       'SoutCantExist', 'SoutCantInv', 'SoutCantStaple', 'SoutCantTrans',
       'SoutMontoCedis', 'SoutMontoCteDesp', 'SoutMontoDesp', 'SoutMontoExist',
       'SoutMontoInv', 'SoutMontoStaple', 'SoutMontoTrans'],
      dtype='object')

In [79]:
data_final['TmpID'].unique()

array([20201025, 20201101], dtype=int64)

In [81]:
data_final[data_final['TmpID'] == 20201025]['SoutCantDesp'].sum()

33477.1292

In [97]:
%%time
try:
    connection = mysql.connector.connect(user='admin', 
                                         password='g3N0mmaLABi$ntelligence',
                                         host='genommalab-businessintelligence.cgkb305m9bb6.us-east-1.rds.amazonaws.com',
                                         database='businessintelligence')
    mySql_insert_query = """INSERT INTO Fact_Desplazamiento_SemanalEcu (ProPstID, SucID, TmpID, SoutCantCedis, SoutCantDesp, SoutCantExist, SoutCantInv,  SoutCantStaple, SoutCantTrans, SoutMontoCedis, SoutMontoCteDesp, SoutMontoDesp, SoutMontoExist, SoutMontoInv, SoutMontoStaple, SoutMontoTrans) 
                           VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) """

    cursor = connection.cursor()
    cursor.executemany(mySql_insert_query, data_upgrade)
    connection.commit()
    print(cursor.rowcount, "Record inserted successfully into Laptop table")
    cursor.close()

except mysql.connector.Error as error:
    print("Failed to insert record into Laptop table {}".format(error))

finally:
    if (connection.is_connected()):
        connection.close()
        print("MySQL connection is closed")

88450 Record inserted successfully into Laptop table
MySQL connection is closed
Wall time: 18.3 s
